# 範例 3：串流輸出

像 ChatGPT 一樣，一個字一個字地顯示回應！

## 學習目標
- 了解串流（Streaming）的概念
- 學會處理串流回應
- 理解即時輸出的實作方式

## 為什麼要用串流？
- **更好的使用者體驗**：不用等待完整回應，馬上就能看到結果
- **感覺更快**：雖然總時間差不多，但即時看到輸出感覺更流暢
- **可以提早中斷**：如果回答不是你要的，可以提早停止

## 前置需求
- Ollama 運行中
- 已下載 gpt-oss:120b 模型

## Step 1: 匯入套件

In [ ]:
import requests
import json

## Step 2: 定義串流對話函數

串流的關鍵設定：
- `"stream": True` - 啟用串流模式
- `stream=True` - 在 requests 中啟用串流接收

In [ ]:
def stream_chat(prompt):
    """
    使用串流方式獲得 AI 回應
    回應會一個字一個字地顯示出來
    """

    url = "http://localhost:11434/api/generate"

    data = {
        "model": "gpt-oss:120b",
        "prompt": prompt,
        "stream": True  # 啟用串流模式
    }

    # 使用串流方式發送請求
    response = requests.post(url, json=data, stream=True)

    print("AI：", end="", flush=True)

    # 逐行讀取回應
    for line in response.iter_lines():
        if line:
            # 解析每一行 JSON
            chunk = json.loads(line)

            # 印出這一小段文字（不換行）
            print(chunk["response"], end="", flush=True)

            # 如果完成了，就跳出迴圈
            if chunk.get("done", False):
                break

    print()  # 最後換行

## Step 3: 測試串流輸出

執行下面的儲存格，觀察文字是如何一個一個出現的：

In [ ]:
question = "請寫一首關於程式設計的短詩。"
print(f"問題：{question}")
print("-" * 50)
stream_chat(question)

## Step 4: 比較串流 vs 非串流

讓我們建立一個非串流版本來比較差異：

In [ ]:
import time

def normal_chat(prompt):
    """非串流方式（一次返回完整回應）"""
    url = "http://localhost:11434/api/generate"
    data = {
        "model": "gpt-oss:120b",
        "prompt": prompt,
        "stream": False
    }
    response = requests.post(url, json=data)
    return response.json()["response"]

In [ ]:
test_prompt = "請用三句話介紹 Python 程式語言。"

print("=== 非串流模式 ===")
print("（需要等待完整回應...）")
start = time.time()
result = normal_chat(test_prompt)
end = time.time()
print(f"AI：{result}")
print(f"等待時間：{end - start:.2f} 秒")

In [ ]:
print("\n=== 串流模式 ===")
print("（文字會即時顯示）")
start = time.time()
stream_chat(test_prompt)
end = time.time()
print(f"總時間：{end - start:.2f} 秒")

## 程式碼解析：關鍵技術點

### 1. `flush=True` 的作用
```python
print(chunk["response"], end="", flush=True)
```
- `end=""` - 不自動換行
- `flush=True` - 立即輸出，不等緩衝區滿

### 2. `response.iter_lines()`
逐行讀取串流回應，每一行是一個 JSON 物件

### 3. 檢查完成狀態
```python
if chunk.get("done", False):
    break
```
當收到 `"done": true` 時，表示回應完成

## 練習：自己試試看

In [ ]:
# 試著問一個需要較長回答的問題
# 這樣更能感受到串流的效果

my_question = "請詳細解釋什麼是機器學習，並舉三個生活中的例子。"
print(f"問題：{my_question}")
print("-" * 50)
stream_chat(my_question)

## 重點回顧

1. **串流模式**：設定 `"stream": True` 啟用
2. **即時輸出**：使用 `print(..., end="", flush=True)`
3. **逐行讀取**：使用 `response.iter_lines()` 處理串流資料
4. **完成標記**：檢查 `"done": true` 判斷是否結束

## 下一步

在下一個範例中，我們將學習如何使用系統提示詞，讓 AI 扮演特定角色！